In [53]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

#https://docs.pennylane.ai/en/stable/code/api/pennylane.fourier.coefficients.html

def fourier_squared_distance(list_of_coeffs, param_list):
    """
    Returns the squared l2-distance in Fourier space between a function
    characterized by its Fourier coefficients and the output of the
    quantum model

    Args:
        list_of coeffs (list(float)): A list of seven coefficients
                                      corresponding to the Fourier
                                      coefficients of the function we
                                      want to approximate
        param_list (list(float)): A list of six parameters characterizing
                                  the angles in the trainable circuit.

    Returns: (float): Squared l2-distance between the given function
                      and the output of the quantum model
    """

    dev = qml.device("default.qubit", wires=3)

    # Feel free to define any helper functions, such as subcircuits, here.

    @qml.qnode(dev)
    def circuit(param_list, x):
        """This circuit returns the PauliZ expectation of
        the quantum model in the statement"""
        # Put your code here #
        
        #W1
        qml.RX(param_list[0],wires=0)
        qml.RX(param_list[1],wires=1)
        qml.RX(param_list[2],wires=2)
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 0])
        
        #input data
        qml.RX(x[0],wires=0)
        qml.RX(x[0],wires=1)
        qml.RX(x[0],wires=2)
        
        #W2
        qml.RX(param_list[3],wires=0)
        qml.RX(param_list[4],wires=1)
        qml.RX(param_list[5],wires=2)
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 0])
        
        return qml.expval(qml.PauliZ(0))

    partial_circuit = functools.partial(circuit, param_list)
    
    num_inputs = 1
    degree = 3
    coeffs = qml.fourier.coefficients(partial_circuit, num_inputs, degree)    
    print(coeffs)

    answer = 0
    for i in range(len(list_of_coeffs)):
        temp = np.abs((list_of_coeffs[i] - coeffs[i])**2)
        print(temp)
        answer += temp

    print(answer)
    # Return your final answer here
    return answer

# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output = fourier_squared_distance(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises:
            ``AssertionError`` if the solution output is incorrect in any way.
    """

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-2
    ), "Your calculated squared distance isn't quite right."


test_cases = [['[[-1.12422548e-01,  0.0, 9.47909940e-02, 0.0, 0.0, 9.47909940e-02, 0.0],[2,2,2,3,4,5]]', '0.0036766085933034303'], ['[[-2.51161988e-01, 0.0, 1.22546112e-01, 0.0, 0.0,  1.22546112e-01, 0.0],[1.1,0.3,0.4,0.6,0.8,0.9]]', '0.6538589174369286']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[-1.12422548e-01,  0.0, 9.47909940e-02, 0.0, 0.0, 9.47909940e-02, 0.0],[2,2,2,3,4,5]]'...
[-1.12422548e-01+0.00000000e+00j -4.95635279e-18-1.09039761e-17j
  9.47909940e-02-4.28754510e-02j -2.19318611e-17-1.58603289e-17j
 -2.19318611e-17+1.58603289e-17j  9.47909940e-02+4.28754510e-02j
 -4.95635279e-18+1.09039761e-17j]
1.0044058486357014e-19
1.4346212851047778e-34
0.0018383042966517165
7.325565643857476e-34
7.325565643857476e-34
0.0018383042966517165
1.4346212851047778e-34
0.003676608593303433
Correct!
Running test case 1 with input '[[-2.51161988e-01, 0.0, 1.22546112e-01, 0.0, 0.0,  1.22546112e-01, 0.0],[1.1,0.3,0.4,0.6,0.8,0.9]]'...
[ 4.58229763e-01+0.00000000e+00j -1.26882631e-16-1.34812796e-16j
 -2.96684095e-02+2.28345944e-01j  3.96508223e-17-1.11022302e-16j
  3.96508223e-17+1.11022302e-16j -2.96684095e-02-2.28345944e-01j
 -1.26882631e-16+1.34812796e-16j]
0.5032366560378175
3.4273692071544275e-32
0.07531113069955551
1.3898139353782174e-32
1.3898139353